In [2]:
import openai
import json
import os

In [5]:
openai.api_key  = os.getenv("OPENAI_API_KEY")

def get_current_weather(location, unit="fahrenheit"):
    weather_info = {
        "location": location,
        "temperature": "72",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

def run_conversation():
    messages = [{"role": "user", "content": "What's the weather like in Boston?"}]
    functions = [
        {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        }
    ]

    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo-0613",
        messages=messages,
        functions=functions,
        function_call="auto",
    )
    response_message = response["choices"][0]["message"]
    print("response message")
    print(response_message)

    if response_message.get("function_call"):
        available_functions = {"get_current_weather": get_current_weather}
        function_name = response_message["function_call"]["name"]
        function_to_call = available_functions[function_name]
        function_args = json.loads(response_message["function_call"]["arguments"])
        function_response = function_to_call(
            location=function_args.get("location"),
            unit=function_args.get("unit"),
        )

        messages.append(response_message)
        messages.append(
            {"role": "function", "name": function_name, "content": function_response}
        )
        second_response = openai.ChatCompletion.create(
            model="gpt-3.5-turbo-0613",
            messages=messages,
        )
        return second_response

In [6]:
run_conversation()

response message
{
  "role": "assistant",
  "content": null,
  "function_call": {
    "name": "get_current_weather",
    "arguments": "{\n  \"location\": \"Boston, MA\"\n}"
  }
}


<OpenAIObject chat.completion id=chatcmpl-7eOELrtTJLPNlIsmXqseWjYCYMR1X at 0x2284c01cef0> JSON: {
  "id": "chatcmpl-7eOELrtTJLPNlIsmXqseWjYCYMR1X",
  "object": "chat.completion",
  "created": 1689860617,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The current weather in Boston, MA is 72\u00b0F. It is sunny and windy."
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 72,
    "completion_tokens": 18,
    "total_tokens": 90
  }
}

In [14]:
import inspect
hello_world = lambda x: "hello world"
inspect.signature(hello_world)
import numpy as np
inspect.signature(np.log)

ValueError: callable <ufunc 'log'> is not supported by signature

In [15]:
np.log.types

['e->e',
 'f->f',
 'd->d',
 'f->f',
 'd->d',
 'g->g',
 'F->F',
 'D->D',
 'G->G',
 'O->O']